## Данные

In [ ]:
!pip install allennlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 23.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
 

In [ ]:
from allennlp.modules.elmo import batch_to_ids, Elmo
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import pandas as pd
import re
import shutil
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/dataset_probe.xlsx')

In [ ]:
def get_elmo_vecs(content):
    splited_tokens = [[w.lower() for w in text_elem.split()] for text_elem in content]
    character_ids = batch_to_ids(content)
    embeddings = elmo(character_ids)['elmo_representations'][0]
    embeddings = embeddings.detach().numpy()

    v_embeds = []
    for idxs, sentence in enumerate(splited_tokens):
        for idxw, word in enumerate(sentence):
            if word == "в":
                v_embeds.append(embeddings[idxs][idxw])
    elmo_X = np.array(v_embeds)
    return elmo_X, v_embeds

In [ ]:
def kmeans_clustering(n, matrix, slcted_columns):    
    kmeans = KMeans(n_clusters=n, init = "random", n_init = "auto", max_iter = 500, algorithm='elkan').fit(matrix)
    df_res = slcted_columns.copy()
    kclusters = kmeans.predict(matrix)
    df_res["clusters"] = kclusters
    return df_res, kclusters

## ruwikiruscorpora_tokens_elmo_1024_2019

In [ ]:
# os.mkdir("ruswikicorp_tokens")
!unzip /content/drive/MyDrive/195.zip

Archive:  /content/drive/MyDrive/195.zip
  inflating: meta.json               
  inflating: model.hdf5              
  inflating: options.json            
  inflating: README                  
  inflating: vocab.txt               


In [ ]:
options_file = f'options.json'
weight_file = f'model.hdf5'
elmo = Elmo(options_file, weight_file, 1)

In [ ]:
new_df, embeds = get_elmo_vecs(df["контексты"])
df["embeds"] = embeds
X_ = np.array(embeds)

In [ ]:
results, cls = kmeans_clustering(4, X_, df[["контексты","значение"]])
results["clusters"] = cls

## ruwikiruscorpora_lemmas_elmo_1024_2019

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/196.zip

--2023-05-30 01:01:17--  http://vectors.nlpl.eu/repository/20/196.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206986351 (197M) [application/zip]
Saving to: ‘196.zip’

196.zip             100%[===================>] 197.40M  10.4MB/s    in 21s     

2023-05-30 01:01:39 (9.43 MB/s) - ‘196.zip’ saved [206986351/206986351]



In [ ]:
!unzip 196.zip

Archive:  196.zip
  inflating: meta.json               
  inflating: model.hdf5              
  inflating: options.json            
  inflating: README                  
  inflating: vocab.txt               


In [ ]:
# os.mkdir("ruswikicorp_lemmas")
# shutil.unpack_archive("/content/drive/MyDrive/196.zip", "/content/ruswikicorp_lemmas")

In [ ]:
options_file = f'options.json'
weight_file = f'model.hdf5'
elmo = Elmo(options_file, weight_file, 1)

In [ ]:
new_df, embeds = get_elmo_vecs(df["контексты"])
df["embeds"] = embeds
X_ = np.array(embeds)

In [ ]:
results, cls = kmeans_clustering(4, X_, df[["контексты","значение"]])
results["clusters"] = cls

In [ ]:
results.loc[results["clusters"] == 0]

,контексты,значение,clusters
0,"В универе очень интересно течет жизнь, постоя...",место,0
2,В Госдуме тоже пытаются бороться с курением,место,0
5,"В книге реально описаны события, которые вызы...",место,0
6,"В ванной было так жарко, что я сделался красн...",место,0
9,"Ты когда-нибудь смотрел в цирке не на манеж, а...",место,0
11,Отвечай на мой запрос в анкете!,место,0
13,В корпорации полностью отказались от производс...,место,0
14,"Вот уже у Бориса в чашке не имбирный чай, а ко...",место,0
15,В Галактике и ее окрестностях оно было предмет...,место,0
19,В модели определены три значения: уязвимость ...,место,0


In [ ]:
results.loc[results["clusters"] == 1]

,контексты,значение,clusters
8,"Кроме того, в фильме не только про этих герое...",место,1
16,Еще более низкие давления бывают в центрах тор...,место,1
21,Смотрю -- в дневнике действительно ничего не н...,место,1
27,"Подскочив от неожиданности, директор пришёл в ...",состояние,1
49,"Несколько мгновений девочка молчала, в замешат...",состояние,1
62,"Жильцы прознали, что, отчаявшись, Рита вызвала...",Профессия,1
65,Джордан пригласил в соавторы двух специалистов...,Профессия,1
74,Поступил в ученики к одному отшельнику и сорок...,Профессия,1
91,Элю с самого нежного возраста куда-то отправля...,Направление,1
98,"Молодая девица смотрела в окно, она ждала весны.",Направление,1


In [ ]:
results.loc[results["clusters"] == 2]

,контексты,значение,clusters
26,"Ксюхины родители в ярости были, но, поскольку ...",состояние,2
46,На следующее утро после выборов Дэвид Кэмерон ...,состояние,2
56,Евфросинией были пострижены в монахини сестра ...,Профессия,2


In [ ]:
results.loc[results["clusters"] == 3]

,контексты,значение,clusters
1,Растить в душе побег уныния -- преступление.,место,3
3,Я старалась подольше оставаться в саду,место,3
4,"Необходимо добавить, что в Китае банковский п...",место,3
7,С приходом тепла в домах стали появляться ванн...,место,3
10,"Зато в Филадельфии, Лондоне, Сан-Франциско, ...",место,3
12,"Рыбу варить в кипятке 10 минут, затем вынуть.",место,3
17,"Но когда холодно, мы и в баре кино смотрим.",место,3
18,Однако в разнообразии таится и некоторая опасн...,место,3
23,Многие хрупкие вещи с полок в кабинетах тогда ...,место,3
25,Друзья в ужасе замахали руками: это совершенно...,состояние,3


##tayga_lemmas_elmo_2048_2019

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/199.zip

--2023-05-30 00:20:02--  http://vectors.nlpl.eu/repository/20/199.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1800442066 (1.7G) [application/zip]
Saving to: ‘199.zip’

199.zip             100%[===================>]   1.68G  10.1MB/s    in 2m 52s  

2023-05-30 00:22:56 (9.99 MB/s) - ‘199.zip’ saved [1800442066/1800442066]



In [ ]:
!unzip 199.zip

Archive:  199.zip
  inflating: meta.json               
  inflating: model.ckpt.data-00000-of-00001  
  inflating: model.ckpt.index        
  inflating: model.ckpt.meta         
  inflating: model.hdf5              
  inflating: options.json            
  inflating: README                  
  inflating: vocab.txt               


In [ ]:
options_file = f'options.json'
weight_file = f'model.hdf5'
elmo = Elmo(options_file, weight_file, 1)

In [ ]:
new_df, embeds = get_elmo_vecs(df["контексты"])
df["embeds"] = embeds
X_ = np.array(embeds)

In [ ]:
results, cls = kmeans_clustering(4, X_, df[["контексты","значение"]])
results["clusters"] = cls

In [ ]:
results.loc[results["clusters"] == 0]

,контексты,значение,clusters
0,"В универе очень интересно течет жизнь, постоя...",место,0
1,Растить в душе побег уныния -- преступление.,место,0
2,В Госдуме тоже пытаются бороться с курением,место,0
5,"В книге реально описаны события, которые вызы...",место,0
6,"В ванной было так жарко, что я сделался красн...",место,0
7,С приходом тепла в домах стали появляться ванн...,место,0
9,"Ты когда-нибудь смотрел в цирке не на манеж, а...",место,0
10,"Зато в Филадельфии, Лондоне, Сан-Франциско, ...",место,0
13,В корпорации полностью отказались от производс...,место,0
14,"Вот уже у Бориса в чашке не имбирный чай, а ко...",место,0


In [ ]:
results.loc[results["clusters"] == 1]

,контексты,значение,clusters
3,Я старалась подольше оставаться в саду,место,1
23,Многие хрупкие вещи с полок в кабинетах тогда ...,место,1
29,После стаканчика вина и в депрессии он описал ...,состояние,1
30,"Сказал, что сам оказался в шоке, узнав, что то...",состояние,1
32,"Неужели ты думала, я буду негодовать на тебя, ...",состояние,1
34,"Я старалась изменить себя и была в восторге, к...",состояние,1
39,Без женского общества Бурцев впадал в уныние,состояние,1
55,"Сле́дующий кандида́т в ректоры, Леони́д Мельни...",Профессия,1
58,Если мальчик из хорошей еврейской семьи не игр...,Профессия,1
63,"Дело новое, конкуренции нет, а завтра, глядишь...",Профессия,1


In [ ]:
results.loc[results["clusters"] == 2]

,контексты,значение,clusters
4,"Необходимо добавить, что в Китае банковский п...",место,2
11,Отвечай на мой запрос в анкете!,место,2
12,"Рыбу варить в кипятке 10 минут, затем вынуть.",место,2
17,"Но когда холодно, мы и в баре кино смотрим.",место,2
18,Однако в разнообразии таится и некоторая опасн...,место,2
25,Друзья в ужасе замахали руками: это совершенно...,состояние,2
33,Публика пришла в восторг,состояние,2
37,Из командировки на Урал Борис Семенович Хитрук...,состояние,2
40,"Однажды я приехала, а она в унынии.",состояние,2
41,Но сначала он пришел в изумление.,состояние,2


In [ ]:
results.loc[results["clusters"] == 3]

,контексты,значение,clusters
8,"Кроме того, в фильме не только про этих герое...",место,3
16,Еще более низкие давления бывают в центрах тор...,место,3
21,Смотрю -- в дневнике действительно ничего не н...,место,3
26,"Ксюхины родители в ярости были, но, поскольку ...",состояние,3
27,"Подскочив от неожиданности, директор пришёл в ...",состояние,3
28,"Был в панике, пока не догадался обратиться к к...",состояние,3
49,"Несколько мгновений девочка молчала, в замешат...",состояние,3
62,"Жильцы прознали, что, отчаявшись, Рита вызвала...",Профессия,3
65,Джордан пригласил в соавторы двух специалистов...,Профессия,3
74,Поступил в ученики к одному отшельнику и сорок...,Профессия,3


##araneum_lemmas_elmo_2048_2020

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/212.zip

--2023-05-30 00:44:04--  http://vectors.nlpl.eu/repository/20/212.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1635573390 (1.5G) [application/zip]
Saving to: ‘212.zip’

212.zip             100%[===================>]   1.52G  10.5MB/s    in 2m 35s  

2023-05-30 00:46:40 (10.0 MB/s) - ‘212.zip’ saved [1635573390/1635573390]



In [ ]:
!unzip 212.zip

Archive:  212.zip
  inflating: meta.json               
  inflating: model.ckpt.data-00000-of-00001  
  inflating: model.ckpt.index        
  inflating: model.ckpt.meta         
  inflating: model.hdf5              
  inflating: options.json            
  inflating: README                  
  inflating: vocab.txt               


In [ ]:
options_file = f'options.json'
weight_file = f'model.hdf5'
elmo = Elmo(options_file, weight_file, 1)

In [ ]:
new_df, embeds = get_elmo_vecs(df["контексты"])
df["embeds"] = embeds
X_ = np.array(embeds)

In [ ]:
results, cls = kmeans_clustering(4, X_, df[["контексты","значение"]])
results["clusters"] = cls

In [ ]:
results.loc[results["clusters"] == 0]

,контексты,значение,clusters
1,Растить в душе побег уныния -- преступление.,место,0
3,Я старалась подольше оставаться в саду,место,0
4,"Необходимо добавить, что в Китае банковский п...",место,0
7,С приходом тепла в домах стали появляться ванн...,место,0
8,"Кроме того, в фильме не только про этих герое...",место,0
9,"Ты когда-нибудь смотрел в цирке не на манеж, а...",место,0
10,"Зато в Филадельфии, Лондоне, Сан-Франциско, ...",место,0
11,Отвечай на мой запрос в анкете!,место,0
12,"Рыбу варить в кипятке 10 минут, затем вынуть.",место,0
14,"Вот уже у Бориса в чашке не имбирный чай, а ко...",место,0


In [ ]:
results.loc[results["clusters"] == 1]

,контексты,значение,clusters
16,Еще более низкие давления бывают в центрах тор...,место,1
21,Смотрю -- в дневнике действительно ничего не н...,место,1
23,Многие хрупкие вещи с полок в кабинетах тогда ...,место,1
27,"Подскочив от неожиданности, директор пришёл в ...",состояние,1
29,После стаканчика вина и в депрессии он описал ...,состояние,1
37,Из командировки на Урал Борис Семенович Хитрук...,состояние,1
39,Без женского общества Бурцев впадал в уныние,состояние,1
49,"Несколько мгновений девочка молчала, в замешат...",состояние,1
55,"Сле́дующий кандида́т в ректоры, Леони́д Мельни...",Профессия,1
56,Евфросинией были пострижены в монахини сестра ...,Профессия,1


In [ ]:
results.loc[results["clusters"] == 2]

,контексты,значение,clusters
42,Тут она наткнулась на что-то колючее и отшатну...,состояние,2
57,Все три женщины были взяты в заложники вместе ...,Профессия,2
66,Некрасов каким-то зигзагом оказался на партийн...,Профессия,2
73,Команда корфовской школы вышла в призеры краев...,Профессия,2
75,закину сумку и пойду в универ,Направление,2
81,"Не так давно мама призналась, что после основн...",Направление,2
95,Во время сильной грозы молния ударила в склад ...,Направление,2
99,"Он допил свое виски и сидел, глядя в стол.",Направление,2


In [ ]:
results.loc[results["clusters"] == 3]

,контексты,значение,clusters
0,"В универе очень интересно течет жизнь, постоя...",место,3
2,В Госдуме тоже пытаются бороться с курением,место,3
5,"В книге реально описаны события, которые вызы...",место,3
6,"В ванной было так жарко, что я сделался красн...",место,3
13,В корпорации полностью отказались от производс...,место,3
15,В Галактике и ее окрестностях оно было предмет...,место,3
19,В модели определены три значения: уязвимость ...,место,3
20,"В ячейках, находящихся на пересечении строк и...",место,3
22,В дверях стоял наш папа.,место,3
24,В статье рассматриваются вопросы использования...,место,3
